# Insights-based OVS Flow Visualization PoC


In [1]:
# A bit of styling sugar
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## INPUT

In [2]:
#SOS_REPORT_PATH="/home/amorenoz/dev/sosreports/sosreport-tvhpmc1-pctl002-02710209-2020-07-24-unymyvy"
SOS_REPORT_PATH= "/home/amorenoz/dev/sosreports/sosreport-actlr2-1838-Jan22-2021-2021-01-22-nvjaddl"

## Insights OVN/OVS helpers

In [3]:
from insights.specs import Specs
from insights import Parser, parser
from insights.specs.sos_archive import SosSpecs
from insights.core.context import SosArchiveContext
from insights.core.spec_factory import first_file, RegistryPoint
from insights.parsers import SkipException



import json
import six

class OVNSpecs(Specs):
    ovnnb_db = first_file(["/var/lib/openvswitch/ovn/ovnnb_db.db",
                           "/usr/local/etc/openvswitch/ovnnb_db.db",
                           "/etc/openvswitch/ovnnb_db.db",
                           "/var/lib/openvswitch/ovnnb_db.db"], context=SosArchiveContext)
    
    ovnsb_db = first_file(["/var/lib/openvswitch/ovn/ovnsb_db.db",
                           "/usr/local/etc/openvswitch/ovnsb_db.db",
                           "/etc/openvswitch/ovnsb_db.db",
                           "/var/lib/openvswitch/ovnsb_db.db"], context=SosArchiveContext)

    
## Expects the db file to have been compressed using ovsdb-tool
# ovsdb-tool compact ovnnb_db.db
class OVSDBParser(Parser):
    def __init__(self, *args, **kwargs):
         self._db= {}
         self._schema = {}
        
         super(OVSDBParser, self).__init__(*args, **kwargs)
    
    def parse_content(self, content):
        if not content:
            raise SkipException("Empty Content!")
            
        for line in content:
            if self._is_header(line):
                continue
            
            data = {}
            
            try:
                data = json.loads(line)
            except:
                tb = sys.exc_info()[2]
                cls = self.__class__
                name = ".".join([cls.__module__, cls.__name__])
                msg = "%s couldn't parse json." % name
                six.reraise(ParseException, ParseException(msg), tb)
            
            if self._is_schema(data):
                self._schema = data
            else:
                self._db = data
            
        if not self._db or not self._schema:
            raise SkipException('Invalid Content!')
                
    @property
    def schema(self):
        return self._schema
    
    @property
    def db(self):
        return self._db

    def _is_schema(self, data):
        return 'cksum' in data
        
    def _is_header(self, line):
        return "OVSDB JSON" in line


@parser(OVNSpecs.ovnnb_db)
class OVNNBParser(OVSDBParser):
    def __init__(self, *args, **kwargs):
        super(OVNNBParser, self).__init__(*args, **kwargs)

        
@parser(OVNSpecs.ovnsb_db)
class OVNSBParser(OVSDBParser):
    def __init__(self, *args, **kwargs):
        super(OVNSBParser, self).__init__(*args, **kwargs)
    
    


In [4]:
from insights.core.context import SosArchiveContext
from insights.parsers.ovs_ofctl_dump_flows import OVSofctlDumpFlows
from insights.core import dr


ctx = SosArchiveContext(root=SOS_REPORT_PATH)
broker = dr.Broker()
broker[SosArchiveContext] = ctx
broker = dr.run(broker=broker)

- Add missing ovs_ofctl_dump_flows to SosArchiveContext

In [5]:
#from insights.specs import Specs
#from insights.core.spec_factory import glob_file
#from insights.specs.sos_archive import SosSpecs

#class SosSpecs2(Specs):
#    ovs_ofctl_dump_flows = glob_file("/sos_commands/openvswitch/ovs-ofctl_dump-flows*", context=SosArchiveContext)

## Visualization helpers

In [6]:
# QGrid seems like a good way to show interactive tables
import pandas as pd

import qgrid
def show_table(dframe):
    ## Drop columns with all missing values
    ## Todo: readonly
    return qgrid.show_grid(dframe.dropna(axis=1, how='all'))


## Data extraction

In [7]:
# OVN
ovn_nb = {}
for table, content in broker[OVNNBParser].db.items():
    if table in ["_date", "_comment"]:
        continue
    ovn_nb[table] = pd.DataFrame.from_dict(content, orient='index')

ovn_sb = {}
for table, content in broker[OVNSBParser].db.items():
    if table in ["_date", "_comment"]:
        continue
    ovn_sb[table] = pd.DataFrame.from_dict(content, orient='index')

# OVS
bridges = [parser.bridge_name for parser in broker[OVSofctlDumpFlows]]
ofdumps = { bridge: pd.DataFrame(list(filter(lambda x: x.bridge_name == bridge, broker[OVSofctlDumpFlows]))[0].flow_dumps)
         for bridge in bridges
}    

## Visualization

In [8]:
import string
import numpy
from ipywidgets import interact, interact_manual
import ipywidgets as widgets

### OVS

In [12]:
pd.set_option('display.max_colwidth', None)
selected = widgets.Output(layout={'border': '1px solid black'})

def show_details(event, qgrid):
    output_area = selected
    
    with output_area:
        row = qgrid.get_selected_df().dropna(axis=1, how='all').T
        
        display(row)
        output_area.clear_output(wait=True)

        
bridge_sel = widgets.Dropdown (
    options = bridges,
    description = "Bridge:"
)

table_sel = widgets.Dropdown(
    options=[0],
    value=0,
    description='Table:',
    disabled=False,
)

priority_sel = widgets.Dropdown(
    options=[-1],
    value=-1,
    description='Priority:',
    disabled=False,
)

action_sel = widgets.Text(value='',
                          placeholder='',
                          description='Action match:',
                          disabled=False)

in_sel = widgets.Text(value='',
                      placeholder='',
                      description='In port match:',
                      disabled=False)

## Make table and priority selector show the options based on bridge
def set_table_and_prio_options(change):
    bridge = change["new"]
    with bridge_sel.hold_trait_notifications():
        table_sel.options = numpy.append(-1, ofdumps[bridge]['table'].unique())
        table_sel.value=0
        priority_sel.options=numpy.append(-1, ofdumps[bridge]['priority'].unique())
        
        
bridge_sel.observe(set_table_and_prio_options, 'value')
        
@interact
def show_flows(bridge=bridge_sel,
               table=table_sel,
               priority=priority_sel,
               action=action_sel,
               in_port=in_sel):

    flows = ofdumps[bridge]
    global flow_grid
    ncols = len(flows.index)

    query = []
    if table >= 0:
        query.append("table == {}".format(table))
    if priority >= 0:
        query.append("priority == {}".format(priority))

    if action:
        _actions = flows['actions'].str.match(action) == True
    else:
        _actions = pd.Series([True for i in range(ncols)])

    if in_port:
        _in_port = flows['in_port'].astype('string').str.match(in_port) == True
    else:
        _in_port = pd.Series([True for i in range(ncols)])

    _flows = flows[_actions & _in_port]

    if query:
        flow_grid = show_table(_flows.query(" & ".join(query)))
    else:
        flow_grid = show_table(_flows)

    flow_grid.on('selection_changed', show_details)

    return flow_grid



interactive(children=(Dropdown(description='Bridge:', options=('br-ex', 'br-int', 'br-private'), value='br-ex'…

In [13]:
selected

Output(layout=Layout(border='1px solid black'))

### OVN

In [14]:

def show_ovn(db):
    table_sel = widgets.Dropdown(
        options = db.keys(),
        description= "Table:"
    )
    ## We could automatically generate filter-widgets based on database schema
    @interact
    def show_flows(table=table_sel):
        ##
        return show_table(db[table])

show_ovn(ovn_nb)
show_ovn(ovn_sb)


interactive(children=(Dropdown(description='Table:', options=('Logical_Router_Static_Route', 'NB_Global', 'ACL…

interactive(children=(Dropdown(description='Table:', options=('RBAC_Permission', 'RBAC_Role', 'SB_Global', 'Po…